### Introduction to Route Analysis using Batfish

Network engineers routinely need to verify the routing protocol information in the network. This involves connecting to the various network devices and executing a series of `show` commands. In a multi-vendor network, network engineers must remember the exact command syntax and result formats, making this task very complex. 

However, with Batfish and the vendor-independent models that it produces, this task is extremely simple.

In this notebook, we will look at how you can extract routing protocol-related information from Batfish.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-route-analysis?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py

 ### Initializing our Network and Snapshot

`SNAPSHOT_PATH` below can be updated to point to a custom snapshot directory, see the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis.<br>
More example networks are available in the [networks](https://github.com/batfish/batfish/tree/master/networks) folder of the Batfish repository.

In [2]:
# Initialize a network and snapshot
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

The network snapshot that we initialized above is illustrated below. You can download/view devices' configuration files [here](https://github.com/batfish/batfish/tree/master/test_rigs/example).

![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/example/example-network.png)

All of the information we will show you in this notebook is dynamically computed by Batfish based on the configuration files for the network devices.

### Exploring routes computed by Batfish

Let's start with listing all of the Layer 3 (IP) neighbors of the router **as2border2**.

Questions that provide a `.frame()` method return a Pandas Dataframe. To learn more about Pandas, follow this [link](https://pandas.pydata.org/pandas-docs/stable/tutorials.html).

In [3]:
answer = bfq.neighbors(node1Regex="as2border2", neighborTypes=["layer3"]).answer()['answerElements'][0]
# Information about the Layer 3 neighbors is present under the key layer3Neighbors in the answer
pd.DataFrame(answer['layer3Neighbors'])

,node1,node1interface,node2,node2interface
0,as2border2,GigabitEthernet0/0,as3border1,GigabitEthernet1/0
1,as2border2,GigabitEthernet1/0,as2core2,GigabitEthernet0/0
2,as2border2,GigabitEthernet2/0,as2core1,GigabitEthernet1/0


Now let's get the list of VRFs configured on **as2border2** and the interfaces associated with each of them.

In [4]:
interface_vrf = bfq.interfaceProperties(nodeRegex="as2border2", propertySpec="vrf|interface").answer().frame()
interface_vrf

,interface,vrf
0,as2border2:GigabitEthernet1/0,default
1,as2border2:GigabitEthernet2/0,default
2,as2border2:Ethernet0/0,default
3,as2border2:Loopback0,default
4,as2border2:GigabitEthernet0/0,default


In the example network, no VRFs have been configured, so all interfaces are part of the **default** VRF. This is also referred to as the **global VRF** or **global table** by some vendors and network engineers.

Let's look at the routing table for the **default** VRF on **as2border2**.

In [5]:
routing_table = bfq.routes(nodeRegex="as2border2", vrfRegex="default").answer().frame()
routing_table

,Node,VRF,Network,Protocol,Tag,NextHopIp,NextHop,AdminDistance,Metric
0,as2border2,default,2.1.3.1/32,ospf,-1,2.12.22.2,as2core2,110,3
1,as2border2,default,2.1.3.1/32,ospf,-1,2.12.21.2,as2core1,110,3
2,as2border2,default,10.23.21.2/32,local,-1,AUTO/NONE(-1l),None,0,0
3,as2border2,default,2.1.2.2/32,ospf,-1,2.12.22.2,as2core2,110,2
4,as2border2,default,2.12.12.0/24,ospf,-1,2.12.22.2,as2core2,110,2
5,as2border2,default,3.0.1.0/24,bgp,-1,10.23.21.3,as3border1,20,50
6,as2border2,default,2.23.21.0/24,ospf,-1,2.12.22.2,as2core2,110,2
7,as2border2,default,2.12.21.1/32,local,-1,AUTO/NONE(-1l),None,0,0
8,as2border2,default,2.128.0.0/16,aggregate,-1,AUTO/NONE(-1l),None,200,0
9,as2border2,default,1.0.2.0/24,ibgp,-1,10.12.11.1,as1border1,200,50


Often times, network engineers will check the routing tables for multiple devices in order to chase down a connectivity issue.

So now, let's find all devices that have a route for an arbitrary network with prefix **1.0.2.0/24**.

We will use the pandas [unique](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html) function to filter out the duplicates from the [pandas Series](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html) returned from `filtered_routes["Node"]`.

In [6]:
# Getting routes satisfying our condition
all_routes = bfq.routes().answer().frame()
filtered_routes = all_routes[all_routes["Network"] == "1.0.2.0/24"]

# Getting the sorted and unique node names from the result
print(sorted(filtered_routes["Node"].unique()))

['as1border1', 'as1border2', 'as1core1', 'as2border1', 'as2border2', 'as2core1', 'as2core2', 'as2dept1', 'as2dist1', 'as2dist2', 'as3border1', 'as3border2', 'as3core1']


Figuring out which routers have routes to a given prefix installed is useful, but we also want to know which routers do not have this prefix.<br>
So let's give that a try as well.

We will be using pandas [groupby](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) function to operate on group of routes belonging to the same router and apply [filter](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.filter.html) on all routes from that group.

In [7]:
# First let's get all the routes present in the network
all_routes = bfq.routes().answer().frame()

# Grouping the result by Node and getting only the group which don't have Network == 1.0.2.0/24
filtered_groups = all_routes.groupby('Node').filter(lambda x: all(x['Network'] != '1.0.2.0/24'))

# Getting the sorted and unique node names from the result
print(sorted(filtered_groups["Node"].unique()))

['host1', 'host2']


In this case, all of the routers have this prefix. Only the 2 servers host1 and host2 do not have this prefix
in their routing table.

In this notebook we showed you:

1. How to retrieve the list of Layer 3 neighbors for any router in the network
2. Retrieve the list of VRFs, and the interfaces in each VRF for any router
3. How to find all routers that have a specific prefix in the routing table
4. How to find all routers that are missing a specific prefix in the routing table

These are foundational concepts for using Batfish to perform route analysis. In future notebooks we will dive into more advanced route analysis using Batfish.

### Want to know more? 

Reach out to us through [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) or [Github](https://github.com/batfish/batfish) to learn more. We would love to hear about your network and talk about how Batfish can help improve its security and reliability.

We hope that you will continue to explore **Batfish** and all that it has to offer.